# Tâche 2 - Modèles de langue N-grammes - Comme le disait le proverbe...

L'objectif de cette tâche est de construire un modèle de langue Ngrammes de proverbes et de l'utiliser pour accomplir 2 sous-tâches. La première consiste à déterminer, parmi les choix que nous vous offrons, lequel correspond à un proverbe connu. Cette épreuve vise à évaluer la capacité des modèles unigrammes, bigrammes et trigrammes à bien estimer la vraisemblance d'un passage de texte.  

La deuxième sous-tâche consiste à utiliser les modèles N-grammes comme des générateurs de textes et à compléter des proverbes. On vous donne une liste de proverbes se terminant par des mots masqués. L'épreuve consiste à générer un nombre de mots équivalent au nombre de masques et à faire la substitution des masques par les mots.

Voir l'énoncé du travail #1 pour une description plus détaillée de cette tâche et des 2 sous-tâches.

Les fichiers à utiliser pour cette tâche :
- *t2_proverbes.txt*: il contient plus de 3000 proverbes, un par ligne de texte. Vous utilisez ce fichier pour l'entraînement des modèles de langues N-grammes.
- *t2_proverbes_test1.json*: il contient des listes choix de réponses, chaque liste contenant un seul proverbe connu. À utiliser pour évaluer la capacité des modèles de langue N-grammes à bien estimer la vraisemblance d'un passage de texte.
- *t2_proverbes_test2.json*: il contient des proverbes se terminant par des mots masqués. À utiliser pour générer des mots avec les modèles N-grammes qui remplaceront les masques.

Consignes:
- Utilisez NLTK pour construire les modèles de langue.
- Utilisez NLTK (par exemple la fonction *word_tokenize*) pour la tokenisation des textes.
- N'oubliez pas de faire le rebourrage (*padding*) des proverbes avec des symboles de début et de fin.
- Faites un lissage de Laplace des modèles.
- Il n’est pas nécessaire d’ajouter un jeton de mot inconnu (<UNK>)
- Ne pas modifier les fonctions *load_proverbs* et *load_tests*.
- Utilisez la variable *models* pour conserver les modèles après entraînement.
- Ne pas modifier la signature de les fonctions *train_models* et *correct_proverb*.
- Des modifications aux signatures de fonctions entraîneront des pénalités dans la correction.
- Vous pouvez ajouter des cellules au *notebook* et ajouter toutes les fonctions utilitaires que vous voulez.

## Section 1 - Lecture des fichiers de données (proverbes et tests)

On définit ici 2 fonctions pour lire le fichier de proverbes et les 2 fichiers de test (un pour chacun des sous-tâches).

In [54]:
import json

# Ne pas modifier le chemin de ces 2 fichiers pour faciliter notre correction
proverbs_fn = "./data/t2_proverbes.txt"
test1_fn = './data/t2_proverbes_test1.json'  # tests d'estimation de vraisemblance
test2_fn = './data/t2_proverbes_test2.json'  # tests de génération de texte


def load_proverbs(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_lines = f.readlines()
    return [x.strip() for x in raw_lines]


def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [55]:
proverbs = load_proverbs(proverbs_fn)

Quelques informations à propos des proberbes:

In [57]:
print("Nombre de proverbes pour l'entraînement: {}".format(len(proverbs)))
print("\nUn exemple de proverbe: " + proverbs[10])

Nombre de proverbes pour l'entraînement: 3108

Un exemple de proverbe: affaire menée sans bruit se fait avec plus de fruit


On monte également les 2 fichiers de test pour visualiser leur contenu.

In [58]:
tests_estimation = load_tests(test1_fn)

In [60]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests_estimation)
pd.set_option("display.max_colwidth", 120)
display(df)

,Candidates,Proverb
0,"[a beau mentir qui vient de loin, a beau partir qui vient de loin, a beau dormir qui vient de loin]",a beau mentir qui vient de loin
1,"[l’occasion fait le larron, l’occasion forge le bon homme, l'argent fait le larron]",l’occasion fait le larron
2,"[pas de soucis, le ciel t’aidera, endors-toi, le ciel t’aidera, aide-toi, le ciel t’aidera, bouge, le ciel est bleu ...","aide-toi, le ciel t’aidera"
3,"[aide-toi, le ciel t’aura, aide-toi, le ciel t’aide, aide-toi, le ciel t’aidera]","aide-toi, le ciel t’aidera"
4,"[ce que femme dit, dieu dit, ce que femme veut, dieu le veut, ce que femme veut, dieu aide et prête la main]","ce que femme veut, dieu le veut"
5,"[bien mal acquis jamais, bien mal acquis ne sait et ne profite jamais, bien mal acquis ne profite jamais]",bien mal acquis ne profite jamais
6,"[bon ouvrier ne travaille pas sans ses outils, bon ouvrier ne déplace pas ses outils, bon ouvrier ne querelle pas se...",bon ouvrier ne querelle pas ses outils
7,"[pour le fou, c’était tous les jours fête, pour le fou, ce sera tous les jours fête, pour le fou, c’est tous les jou...","pour le fou, c’est tous les jours fête"
8,"[dire et dire, sont un, dire et faire, sont deux, dire, faire et plaire, sont trois]","dire et faire, sont deux"
9,"[mieux vaut prévenir que guérir, mieux vaut prévenir, mieux vaut prévenir que courir et guérir]",mieux vaut prévenir que guérir


In [61]:
tests_generation = load_tests(test2_fn)

In [62]:
df = get_dataframe(tests_generation)
pd.set_option("display.max_colwidth", 100)
display(df)

,Masked,Proverb
0,a beau mentir qui vient de ***,a beau mentir qui vient de loin
1,a beau mentir qui vient *** ***,a beau mentir qui vient de loin
2,a beau mentir qui *** *** ***,a beau mentir qui vient de loin
3,l’occasion fait *** ***,l’occasion fait le larron
4,"aide-toi, le ciel t’***","aide-toi, le ciel t’aidera"
5,"ce que femme veut, dieu le ***","ce que femme veut, dieu le veut"
6,"ce que femme veut, dieu *** ***","ce que femme veut, dieu le veut"
7,bien mal acquis ne profite ***,bien mal acquis ne profite jamais
8,bon ouvrier ne querelle pas ses ***,bon ouvrier ne querelle pas ses outils
9,bon ouvrier ne querelle *** *** ***,bon ouvrier ne querelle pas ses outils


## Section 2 - Construction des modèles de langue N-grammes.

La fonction ***train_models*** prend en entrée une liste de proverbes et construit les trois modèles unigramme, bigramme et trigramme.

Les 3 modèles entraînés sont conservés dans ***models***, un dictionnaire Python qui prend la forme

<pre>
{
   1: modele_unigramme,
   2: modele_bigramme,
   3: modele_trigramme
}
</pre>

avec comme clé la valeur N du modèle et comme valeur le modèle construit par NLTK.

Metter dans les prochaines cellules le code pour construire vos modèles avec NLTK, pour obtenir les n-grammes de mots, pour déterminer le vocabulaire, etc...

In [63]:
# Import des modules nécessaires

import nltk
from nltk import word_tokenize
from nltk.util import ngrams, pad_sequence
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import Laplace

In [64]:


BOS = '<BOS>'  # Jeton de début de proverbe
EOS = '<EOS>'  # Jeton de fin de proverbe

models = {}

In [65]:
# Fonction pour construire le vocabulaire

def build_vocabulary(proverbs):
    """
    Construit le vocabulaire (liste des mots distincts) à partir des proverbes.
    On ajoute BOS et EOS explicitement pour être sûrs qu'ils sont inclus.
    """
    all_tokens = []
    for sentence in proverbs:
        tokens = word_tokenize(sentence.lower())
        all_tokens.extend(tokens)
    voc = set(all_tokens)
    voc.add(BOS)
    voc.add(EOS)
    return list(voc)

In [66]:
# Fonction pour obtenir les n-grammes avec padding

def get_ngrams(proverbs, n=2):
    """
    Retourne tous les n-grammes (avec padding) pour un corpus de proverbes.
    """
    all_ngrams = []
    for sentence in proverbs:
        tokens = word_tokenize(sentence.lower())
        padded_sent = list(
            pad_sequence(tokens,
                         pad_left=True, left_pad_symbol=BOS,
                         pad_right=True, right_pad_symbol=EOS,
                         n=n)
        )
        all_ngrams.extend(list(ngrams(padded_sent, n=n)))
    return all_ngrams

In [67]:
# Fonction utilitaire pour pipeline NLTK (on met les données dans la format attendu par NLTK, notamment padding)

def prepare_data(proverbs, n):
    """
    Utilise la fonction padded_everygram_pipeline de NLTK
    pour générer les n-grammes et le vocabulaire associés.
    """
    tokenized = [word_tokenize(p.lower()) for p in proverbs]
    train_data, vocab = padded_everygram_pipeline(n, tokenized)
    return train_data, vocab

In [68]:
#Fonction principale pour entraîner les modèles
def train_models(proverbs):
    """ Vous ajoutez à partir d'ici le code dont vous avez besoin
        pour construire les différents modèles N-grammes.
        Cette fonction doit construire tous les modèles en une seule passe.
        Voir les consignes de l'énoncé du travail pratique concernant les modèles à entraîner.

        Vous pouvez ajouter les fonctions/méthodes et variables que vous jugerez nécessaire.
        Merci de ne pas modifier la signature et le comportement de cette fonction (nom, arguments, sauvegarde des modèles).
    """

    for n in [1, 2, 3]:
        # Prépare les données (ngrams + vocabulaire + padding) pour cet ordre
        train_data, vocab = prepare_data(proverbs, n)

        # Construis le modèle avec Laplace smoothing
        model = Laplace(n)

        # Entraîne le modèle
        model.fit(train_data, vocab)

        # Sauvegarde dans le dictionnaire global
        models[n] = model

In [69]:
# Entraînement des 3 modèles
train_models(proverbs)

In [70]:
# 1. On vérifie qu’on a bien les trois modèles
print("Clés disponibles dans 'models':", models.keys())

# 2. Exemple : probabilité d’un mot avec unigramme
print("\n[Unigramme] P('mieux') =", models[1].score('mieux'))

# 3. Exemple : probabilité conditionnelle avec bigramme
print("\n[Bigramme] P('vaut' | 'mieux') =", models[2].score('vaut', ['mieux']))

# 4. Exemple : probabilité conditionnelle avec trigramme
print("\n[Trigramme] P('prévenir' | 'mieux vaut') =", models[3].score('prévenir', ['mieux','vaut']))

# 5. Vérifier qu’un mot inconnu n’a pas probabilité 0 (Laplace smoothing)
print("\n[Bigramme] P('xyz' | 'mieux') =", models[2].score('xyz', ['mieux']))

# 6. Vérifier perplexité sur une courte séquence connue
test_seq = list(ngrams(['mieux','vaut','prévenir','que','guérir'], n=2))
print("\n[Bigramme] Perplexité sur 'mieux vaut prévenir que guérir' =", models[2].perplexity(test_seq))

Clés disponibles dans 'models': dict_keys([1, 2, 3])

[Unigramme] P('mieux') = 0.003018336393591066

[Bigramme] P('vaut' | 'mieux') = 0.011326508385972555

[Trigramme] P('prévenir' | 'mieux vaut') = 0.00044218439089100157

[Bigramme] P('xyz' | 'mieux') = 0.00021781746896101068

[Bigramme] Perplexité sur 'mieux vaut prévenir que guérir' = 838.1694486228748


Les vérifications montrent que :
- Les trois modèles (unigramme, bigramme, trigramme) ont bien été construits et stockés dans models.
- Les probabilités estimées sont cohérentes :
    Unigrammes → fréquence brute des mots dans le corpus.
    Bigrammes / Trigrammes → probabilités conditionnelles qui diminuent quand le contexte est plus long.
- Le lissage de Laplace fonctionne : même un mot totalement inconnu (xyz) reçoit une probabilité non nulle.
- La perplexité calculée sur un proverbe connu est élevée (≈ 838). Cela reflète le fait que le corpus est vaste et varié : le modèle a du mal à être très confiant sur une courte séquence. On s’attend à ce que la perplexité diminue quand on utilisera les trigrammes, qui capturent plus de contexte.


## Section 3 - Choisir le proverbe connu parmi plusieurs candidats

Expliquez ici comment vous procédez pour choisir une option parmi les candidats proposés.

Pour chaque question du fichier t2_proverbes_test1.json, on dispose :
- d’une liste de phrases candidates,
- d’un proverbe correct.

L’objectif est de sélectionner, à l’aide de nos modèles N-grammes (unigramme, bigramme, trigramme), le candidat qui a la probabilité la plus élevée selon le modèle.

Notre démarche est la suivante :

1 - Tokenisation et padding :
- Chaque candidat est découpé en mots avec word_tokenize.
- On ajoute les symboles `<BOS>` et `<EOS>` selon l’ordre du modèle (1, 2 ou 3).

2 - Calcul de la vraisemblance
- Pour un candidat donné, on décompose sa phrase en n-grammes.
- On multiplie les probabilités des n-grammes successifs fournies par le modèle.

3 - Choix du meilleur candidat
- On compare les scores obtenus pour tous les candidats.
- Celui qui minimise la perplexité (ou maximise la probabilité) est retenu comme le proverbe prédit par le modèle.

4 - Évaluation des performances
- On compare la prédiction avec le proverbe correct donné dans le fichier de test.
- On calcule le taux de succès pour chaque modèle (unigramme, bigramme, trigramme) :
    - La métrique qu’on utilise est l’accuracy
    - Elle mesure simplement la proportion de fois où le modèle choisit le bon proverbe parmi les candidats.
    - cela suffit dans notre cas car chaque test a exactement une seule bonne réponse, il n’y a pas de classes déséquilibrées (chaque test est binaire : correct / incorrect), pas besoin de métriques plus fines comme précision, rappel ou F1 (utiles surtout quand il y a plusieurs classes ou un déséquilibre)

=======================================================================================================================================

On imagine déjà que :
- Les unigrammes se basent uniquement sur la fréquence brute des mots, sans tenir compte de l’ordre → donc peu précis.
- Les bigrammes ajoutent une dépendance au mot précédent, ce qui améliore la cohérence des proverbes.
- Les trigrammes capturent encore plus de contexte et devraient mieux discriminer les candidats.

Avec cette méthode, on peut analyser l’impact de la longueur de l’historique (N=1,2,3) sur la capacité des modèles à reconnaître les proverbes corrects.






Voici les tests à utiliser pour cette sous-tâche. Chaque test contient une liste de candidats et le proverbe connu qui devrait être choisi par le modèle.

In [71]:
print("Nombre de tests:", len(tests_estimation))

Nombre de tests: 21


In [72]:
tests_estimation[0]  # Un exemple de test


{'Candidates': ['a beau mentir qui vient de loin',
  'a beau partir qui vient de loin',
  'a beau dormir qui vient de loin'],
 'Proverb': 'a beau mentir qui vient de loin'}

Complétez la fonction suivante qui fait le choix de l'option qui semble la plus vraisemblable parmi une liste de candidats. Ne pas modifier la signature de la fonction ni ses valeurs de retour.

In [73]:
# Fonction d'évaluation

def select_proverb(candidates, n=3):
    """Retourne le candidat retenu et son score basé sur la perplexité.

    *candidates* : liste de textes candidats (dont un seul est correct)
    *n* : ordre du modèle (1 = unigramme, 2 = bigramme, 3 = trigramme)
    """

    model = models[n]
    best_score = float('inf')  # on cherche la plus faible perplexité
    result = None

    for cand in candidates:
        # Extraire les n-grammes du candidat
        ngrams_cand = get_ngrams([cand], n=n)

        # Calculer la perplexité sur la séquence
        try:
            score = model.perplexity(ngrams_cand)
        except ZeroDivisionError:
            # Cas pathologique si la phrase est très courte ou OOV
            score = float('inf')

        # On choisit le candidat avec la perplexité la plus faible
        if score < best_score:
            best_score = score
            result = cand

    return result, best_score


**REMARQUE**

Initialement, on avait envisagé de faire la sélection du proverbe en sommant les log-probabilités des n-grammes, mais cette méthode favorisait artificiellement les proverbes courts (moins de termes négatifs à additionner) : `score = sum(model.logscore(ng[-1], list(ng[:-1])) for ng in ngrams_cand)`

Nous avons amélioré la fonction en utilisant la perplexité comme critère de choix : c’est une mesure normalisée par la longueur et standard en modélisation de langue, qui permet de comparer équitablement les candidats.

Mettez dans les prochaines cellules le code nécessaire afin d'évaluer chacun des modèles pour cette sous-tâche.

In [74]:
def evaluate_model(tests, n=3):
    """
    Évalue un modèle N-gramme (n = 1, 2 ou 3) sur la sous-tâche 1.
    Retourne le nombre de bonnes réponses, le total et l'accuracy.
    """
    correct = 0
    total = len(tests)

    for test in tests:
        candidates = test["Candidates"]
        gold = test["Proverb"]

        predicted, score = select_proverb(candidates, n=n)

        if predicted == gold:
            correct += 1

    accuracy = correct / total
    return correct, total, accuracy


In [75]:
# Évaluation modèle unigramme (N=1)

correct, total, acc = evaluate_model(tests_estimation, n=1)
print(f"Modèle 1-grammes : {correct}/{total} corrects  (Accuracy = {acc:.2%})")


Modèle 1-grammes : 8/21 corrects  (Accuracy = 38.10%)


In [78]:
# Évaluation modèle bigramme (N=2)

correct, total, acc = evaluate_model(tests_estimation, n=2)
print(f"Modèle 2-grammes : {correct}/{total} corrects  (Accuracy = {acc:.2%})")



Modèle 2-grammes : 15/21 corrects  (Accuracy = 71.43%)


In [77]:
# Évaluation modèle trigramme (N=3)

correct, total, acc = evaluate_model(tests_estimation, n=3)
print(f"Modèle 3-grammes : {correct}/{total} corrects  (Accuracy = {acc:.2%})")



Modèle 3-grammes : 20/21 corrects  (Accuracy = 95.24%)


Présentez ici vos résultats et répondez aux questions formulées dans l'énoncé du travail. Vous pouvez ajouter des cellules au besoin.

### Résultats obtenus





Unigramme : 8/21 corrects → 38.1%

Bigramme : 15/21 corrects → 71.4%

Trigramme : 20/21 corrects → 95.2%

### Analyse






La sélection du proverbe se fait en comparant la perplexité des candidats : celui avec la plus faible perplexité est choisi.

Les performances montrent un écart clair entre les modèles :
- L’unigramme est trop limité : il ne considère que les fréquences individuelles des mots, sans ordre → faible précision.
- Le bigramme capte des associations locales fréquentes et améliore nettement la reconnaissance.
- Le trigramme exploite un contexte plus riche et capture beaucoup mieux la structure proverbiale → précision quasi parfaite.


### Impact de la longueur de l'historique (N)

Donc comme on l'imaginait, plus la longueur de l’historique augmente, plus le modèle reflète fidèlement le langage des proverbes.

**Explications :**
- **N=1 (Unigramme)** : Considère uniquement les fréquences individuelles des mots,
  sans tenir compte de leur ordre → incapable de distinguer une structure proverbiale
  d'une suite aléatoire de mots courants.
  
- **N=2 (Bigramme)** : Capture des associations locales fréquentes → amélioration nette de la reconnaissance des structures typiques.
  
- **N=3 (Trigramme)** : Exploite un contexte plus riche et reconnaît des expressions
  figées complètes → précision quasi parfaite.

**Conclusion :** Plus la longueur de l'historique augmente, plus le modèle reflète
fidèlement le langage des proverbes. Le contexte supplémentaire permet de capturer
les structures caractéristiques.

### Capacité à capturer le langage des proverbes

- **Unigramme** : Capture mal le langage des proverbes (seulement le vocabulaire,
  pas la structure)
  
- **Bigramme** : Capture partiellement (reconnaît des paires de mots typiques mais
  manque les expressions plus longues)
  
- **Trigramme** : Capture très bien le langage des proverbes (reconnaît les
  expressions figées et structures idiomatiques)

Le trigramme reproduit fidèlement les régularités lexicales et syntaxiques des proverbes.
Sa réussite à 95% démontre qu'un contexte de 3 mots suffit à identifier la plupart des
patterns caractéristiques. L'unique erreur suggère que certains proverbes rares ou
ambigus restent difficiles à distinguer.


## Section 4 - Remplacer les masques du proverbe avec des mots générés

Utilisez pour cette sous-tâche les proverbes masqués (dans la variable *tests_generation*) obtenus à la Section 1 à partir du fichier *'./data/t2_proverbes_test2.json'*.

In [79]:
len(tests_generation)

27

In [80]:
tests_generation[:5]

[{'Masked': 'a beau mentir qui vient de ***',
  'Proverb': 'a beau mentir qui vient de loin'},
 {'Masked': 'a beau mentir qui vient *** ***',
  'Proverb': 'a beau mentir qui vient de loin'},
 {'Masked': 'a beau mentir qui *** *** ***',
  'Proverb': 'a beau mentir qui vient de loin'},
 {'Masked': 'l’occasion fait *** ***', 'Proverb': 'l’occasion fait le larron'},
 {'Masked': 'aide-toi, le ciel t’***',
  'Proverb': 'aide-toi, le ciel t’aidera'}]

Compléter le code dans les prochaines cellules pour remplacer les masques par des mots générés. Vous pouvez ajouter des cellules au besoin.

### Génération de proverbes masqués avec modèles N-grammes (Laplace, N=1..3)

Nous utilisons des modèles de langue **N-grammes** (avec lissage **Laplace**) comme **générateurs** pour compléter des proverbes contenant des masques.

#### Idée générale
Étant donné un proverbe masqué (ex. *« a beau mentir qui vient de *** »*), on **remplit les masques `***`** en générant des mots probables selon le **contexte gauche** et, après coup, on **re-classe** les propositions en fonction de la **phrase entière**.

#### Procédure
- **Tokénisation NLTK** et **minuscules** (le corpus d’entraînement est en minuscules).
- **Génération mot par mot** ; pour les **runs de masques consécutifs** (ex. `*** *** ***`), on applique un **beam search court** (top-k/beam) pour explorer quelques suites plausibles.
- On **bloque `<EOS>`** tant qu’il reste des masques à compléter (évite de couper trop tôt).
- On **favorise un “mot de contenu”** pour **terminer un run** (évite de finir par *de/le/la*).
- On réalise ensuite un **re-ranking par perplexité** de la **phrase complète** (BOS/EOS + *everygrams* NLTK) afin de retenir la séquence qui rend **toute** la phrase la plus naturelle selon le modèle.

#### Évaluation
- **Accuracy “phrase”** : exact match entre la prédiction et le gold **après normalisation** (apostrophes/élisions).
- **Accuracy “mots masqués”** (demandée explicitement) : proportion de **mots générés** qui correspondent **aux positions masquées** du proverbe de référence.


In [81]:
# Fonction de normalisation & comparaison par tokens
def tokens_norm(s: str):
    """
    Normalise une chaîne pour la comparaison token-par-token.

    Choix assumé : on *casse* les élisions en remplaçant toute apostrophe par un espace.
    - mise en minuscules ;
    - remplacement des apostrophes typographiques (’ U+2019) et ASCII (') par un espace ;
    - réduction des espaces multiples ;
    - tokenisation NLTK.
    """
    # minuscules
    s = s.lower()
    # remplacer toutes formes d'apostrophe par un espace (on casse les élisions)
    s = s.replace("’", " ").replace("'", " ")
    # normaliser les blancs
    s = re.sub(r"\s+", " ", s).strip()
    # tokeniser
    toks = word_tokenize(s)
    return toks

def eq_by_tokens(pred_str, gold_str):
    """
    Compare deux phrases à l'égalité stricte de leurs séquences de tokens normalisés
    (avec élisions cassées).
    """
    return tokens_norm(pred_str) == tokens_norm(gold_str)



In [82]:
# Utilitaires de génération & perplexité
import re
from nltk.util import pad_sequence, ngrams as _ngrams

def _is_word_like(w: str) -> bool:
    """Mot 'alphabétique' FR (lettres accentuées et tirets intra-mot)."""
    return re.fullmatch(r"[A-Za-zÀ-ÖØ-öø-ÿ]+(?:-[A-Za-zÀ-ÖØ-öø-ÿ]+)*", w) is not None

def _left_context_with_bos(generated, order):
    """Derniers n-1 mots ; si insuffisant (début de phrase), pad avec <BOS>."""
    if order <= 1:
        return []
    need = order - 1
    ctx = generated[-need:]
    if len(ctx) < need:
        ctx = ["<BOS>"] * (need - len(ctx)) + ctx
    return ctx

def sent_ngrams_from_tokens(tokens, n):
    """BOS/EOS + n-grammes de la phrase tokenisée."""
    padded = list(pad_sequence(tokens, pad_left=True, left_pad_symbol="<BOS>",
                               pad_right=True, right_pad_symbol="<EOS>", n=n))
    return list(_ngrams(padded, n=n))

def full_sentence_perplexity(model, tokens):
    """Perplexité de la phrase complète vue par le modèle (ordre = model.order)."""
    seq = sent_ngrams_from_tokens(tokens, model.order)
    return model.perplexity(seq)


In [83]:
# Stoplist: on évite de finir un run par un mot-outil très court
STOP_LAST = {
    "de","du","des","le","la","les","un","une","au","aux","l","d",
    "et","ou","à","en","que","qui","ne","pas","pour","dans","se",
    "ce","ces","leur","leurs","sur","son","sa","ses","y","on","a",
    "est","sont","tu","il","elle","ils","elles","je","me","te","se",
    "plus"
}

def top_k_next_words(model, context, k=8, allow_eos=True, forbid_stop_last=False):
    """
    Retourne les k meilleurs successeurs selon P(.|context), avec filtres.
    """
    forbid = {"<BOS>"}
    if not allow_eos:
        forbid.add("<EOS>")
    cands = []
    for w in model.vocab:
        if w in forbid:
            continue
        if not _is_word_like(w):
            continue
        if forbid_stop_last and (w in STOP_LAST or len(w) <= 2):
            continue
        p = model.score(w, context)
        cands.append((p, w))
    cands.sort(key=lambda x: x[0], reverse=True)
    return [w for _, w in cands[:k]]

def fill_mask_run_with_beam(model, left_generated, run_len, remaining_after,
                            beam_size=3, topk=8):
    """
    Remplit un bloc de `run_len` MASKTOKEN consécutifs par beam search court.
    Renvoie une liste de séquences candidates, ordonnées du meilleur au moins bon
    selon la somme des log-probabilités locales.
    """
    beams = [([], 0.0)]  # (sequence, logP)
    for pos in range(run_len):
        new_beams = []
        for seq, logp in beams:
            # Contexte gauche = déjà généré + séquence partielle du run
            context = _left_context_with_bos(left_generated + seq, model.order)
            # EOS seulement au tout dernier mot du run et s'il ne reste plus de masques après
            allow_eos = (pos == run_len - 1) and (remaining_after == 0)
            # Dernier mot du run: éviter de finir par un mot-outil court
            forbid_stop_last = (pos == run_len - 1)

            for w in top_k_next_words(model, context, k=topk,
                                      allow_eos=allow_eos,
                                      forbid_stop_last=forbid_stop_last):
                p = model.score(w, context)
                lp = math.log(p) if p > 0 else -1e9
                new_beams.append((seq + [w], logp + lp))

        if not new_beams:  # garde-fou
            return [["xxx"] * run_len]

        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:beam_size]

    return [seq for (seq, _) in beams]


In [84]:
# Remplacement des masques
def remove_masks_and_generate(model, masked_text):
    """
    Remplace chaque run de MASKTOKEN par une petite recherche (beam) et
    choisit la meilleure séquence via un re-ranking par perplexité globale.
    Retourne (nb_masks, tokens_générés).
    """
    safe_text = masked_text.lower().replace("***", "MASKTOKEN")
    tokens = word_tokenize(safe_text)

    nb_masks = tokens.count("MASKTOKEN")
    generated = []
    i = 0

    while i < len(tokens):
        if tokens[i] != "MASKTOKEN":
            generated.append(tokens[i])
            i += 1
            continue

        # repérer un run de masques consécutifs
        j = i
        while j < len(tokens) and tokens[j] == "MASKTOKEN":
            j += 1
        run_len = j - i
        remaining_after = tokens[j:].count("MASKTOKEN")

        # candidats via beam
        candidates = fill_mask_run_with_beam(
            model, generated, run_len, remaining_after,
            beam_size=3, topk=8
        )

        # re-ranking par perplexité de la phrase ENTIEREMENT reconstruite
        best_seq, best_pp = None, float("inf")
        for cand in candidates:
            full = generated + cand + tokens[j:]  # gauche + cand + droite
            pp = full_sentence_perplexity(model, full)
            if pp < best_pp:
                best_pp, best_seq = pp, cand

        generated.extend(best_seq if best_seq is not None else candidates[0])
        i = j

    return nb_masks, generated


Mettre dans les prochaines cellules votre code pour évaluer les modèles.

Le sujet demande d’“estimer la proportion de mots générés qui correspondent au proverbe connu (accuracy)”.

On propose donc deux métriques complémentaires :
- Exact match (toute la phrase) — indicatif, mais sévère.
- Accuracy “mots masqués” : on ne note que les mots générés aux positions masquées ; on compte la part correcte.

In [85]:
# Métriques d'évaluation
def masked_word_accuracy(pred_str, gold_str, masked_str):
    """
    Calcule l'accuracy *sur les positions masquées seulement*.
    - On localise les positions des MASKTOKEN dans la version tokenisée du 'masked_str'.
    - On compare, sur ces positions, les tokens normalisés de 'pred_str' et de 'gold_str'.
    Retourne (nb_mask_positions, nb_correct).
    """
    # positions masquées dans la phrase de départ
    masked_tokens = tokens_norm(masked_str.replace("***", "MASKTOKEN")) #les astérisques risquent d’être séparés/jetés comme de la ponctuation
    mask_positions = [i for i, t in enumerate(masked_tokens) if t == "masktoken"]

    # tokens normalisés de pred/gold
    pred_tokens = tokens_norm(pred_str)
    gold_tokens = tokens_norm(gold_str)

    # sécurité : si longueurs différentes, on prend les positions valides
    m = 0
    correct = 0
    for pos in mask_positions:
        if pos < len(pred_tokens) and pos < len(gold_tokens):
            m += 1
            if pred_tokens[pos] == gold_tokens[pos]:
                correct += 1
    return m, correct

import math
def evaluate_generation_exact_and_mask(tests, n=3):
    """
    Évalue un modèle N-grammes (n) pour la complétion :
      - exact match (phrase entière, via eq_by_tokens)
      - accuracy mots masqués
    Retourne un dict de métriques.
    """
    model = models[n]
    total = len(tests)
    exact_correct = 0
    mask_total = 0
    mask_correct = 0

    for test in tests:
        masked = test["Masked"]
        gold = test["Proverb"]
        _, gen_tokens = remove_masks_and_generate(model, masked)
        pred = " ".join(gen_tokens)

        # exact match
        if eq_by_tokens(pred, gold):
            exact_correct += 1

        # mask-only accuracy
        m, c = masked_word_accuracy(pred, gold, masked)
        mask_total += m
        mask_correct += c

    return {
        "exact_correct": exact_correct,
        "total": total,
        "exact_acc": exact_correct/total if total else 0.0,
        "mask_correct": mask_correct,
        "mask_total": mask_total,
        "mask_acc": mask_correct/mask_total if mask_total else 0.0
    }


In [86]:
# résultats globaux
for n in (1, 2, 3):
    m = evaluate_generation_exact_and_mask(tests_generation, n=n)
    print(f"Modèle {n}-grammes :")
    print(f"  Exact match      : {m['exact_correct']}/{m['total']}  (acc = {m['exact_acc']:.2%})")
    print(f"  Mots masqués     : {m['mask_correct']}/{m['mask_total']}  (acc = {m['mask_acc']:.2%})")
    print("-"*60)


Modèle 1-grammes :
  Exact match      : 0/27  (acc = 0.00%)
  Mots masqués     : 3/45  (acc = 6.67%)
------------------------------------------------------------
Modèle 2-grammes :
  Exact match      : 1/27  (acc = 3.70%)
  Mots masqués     : 6/45  (acc = 13.33%)
------------------------------------------------------------
Modèle 3-grammes :
  Exact match      : 15/27  (acc = 55.56%)
  Mots masqués     : 31/45  (acc = 68.89%)
------------------------------------------------------------


In [87]:
# Affichage détaillé
def print_generation_results(tests, n=3, limit=None):
    """
    Affiche pour le modèle n-grammes (n=1/2/3) :
      - Masked (entrée)
      - Pred (prédiction)
      - Gold (référence)
      - Correct ? (exact match, via eq_by_tokens)
    """
    model = models[n]
    total = len(tests)
    to_show = total if limit is None else min(limit, total)

    print(f"\n===== Modèle {n}-grammes =====")
    for i in range(to_show):
        masked = tests[i]["Masked"]
        gold = tests[i]["Proverb"]

        _, gen_tokens = remove_masks_and_generate(model, masked)
        pred = " ".join(gen_tokens)

        ok = eq_by_tokens(pred, gold)
        print(f"Test {i+1}:")
        print(f"  Masked  : {masked}")
        print(f"  Pred    : {pred}")
        print(f"  Gold    : {gold}")
        print(f"  Correct ? {ok}")
        print("-" * 60)

# Exemple : tout afficher
print_generation_results(tests_generation, n=1)
print_generation_results(tests_generation, n=2)
print_generation_results(tests_generation, n=3)



===== Modèle 1-grammes =====
Test 1:
  Masked  : a beau mentir qui vient de ***
  Pred    : a beau mentir qui vient de fait
  Gold    : a beau mentir qui vient de loin
  Correct ? False
------------------------------------------------------------
Test 2:
  Masked  : a beau mentir qui vient *** ***
  Pred    : a beau mentir qui vient de fait
  Gold    : a beau mentir qui vient de loin
  Correct ? False
------------------------------------------------------------
Test 3:
  Masked  : a beau mentir qui *** *** ***
  Pred    : a beau mentir qui de de fait
  Gold    : a beau mentir qui vient de loin
  Correct ? False
------------------------------------------------------------
Test 4:
  Masked  : l’occasion fait *** ***
  Pred    : l ’ occasion fait de fait
  Gold    : l’occasion fait le larron
  Correct ? False
------------------------------------------------------------
Test 5:
  Masked  : aide-toi, le ciel t’***
  Pred    : aide-toi , le ciel t ’ fait
  Gold    : aide-toi, le ciel t’aide

Décrivez les résultats obtenus et répondez aux questions dans l'énoncé du travail. Vous pouvez ajouter le nombre de cellules que vous souhaitez.

### Résultats obtenus




**Exact match (phrase complète)**
- 1-gramme : 0/27 (0.00 %)
- 2-grammes : 1/27 (3.70 %)
- 3-grammes : 15/27 (55.56 %)

**Accuracy “mots masqués”**

Donne une vision plus fine de la capacité à récupérer les bons mots aux positions masquées, même si toute la phrase n’est pas identique
- 1-gramme : 3/45  (acc = 6.67%)
- 2-grammes : 6/45  (acc = 13.33%)
- 3-grammes : 31/45  (acc = 68.89%)



### Analyse





Les résultats montrent un contraste très net entre les trigrammes et les deux autres ordres :
- avec N=3, plus d’une phrase sur deux est reconstruite exactement et environ deux tiers des mots masqués sont corrects, alors que les unigrammes et bigrammes restent très faibles.
- Ce comportement n’est ni accidentel ni “buggué” : il découle directement de la manière dont les N-grammes estiment les probabilités dans un corpus petit et très formulairisé comme un recueil de proverbes.

Commençons par expliquer pourquoi l’unigramme propose des mots “vides” ou inadaptés :
- Un modèle à N=1 ne conditionne jamais sur le contexte : la probabilité d’un mot ne dépend que de sa fréquence globale dans tout le corpus. Sur un ensemble de proverbes, cette distribution est dominée par des mots-outils – prépositions, déterminants, auxiliaires – parce qu’ils apparaissent partout, dans presque toutes les phrases. Quand on demande à l’unigramme de “deviner” un masque dans un contexte tel que « vient de ___ », il n’a aucun moyen de savoir que la suite attendue est “loin” plutôt que “fait” ou “plus” : pour lui, ces trois mots se comparent uniquement par leur fréquence totale. Le mécanisme de lissage de Laplace, que l’on utilise pour éviter les zéros de probabilité, accentue encore cette tendance en redistribuant une petite masse vers tous les mots non vus, ce qui a pour effet secondaire d’aplatir les contrastes et de rapprocher encore davantage le modèle d’un “prior de fréquence” global. Le résultat est logique : on obtient très souvent des sorties grammaticalement stables à l’échelle du corpus (des mots fréquents) mais localement absurdes dans la phrase. D’où les fins du type « … de fait » ou, si l’on ne filtre pas, « … de de ».

- Le bigramme ajoute une information : le prochain mot est conditionné sur le mot précédent. En théorie, cela devrait déjà aider. En pratique, l’amélioration reste modeste pour deux raisons. D’abord, un seul mot de contexte laisse subsister une ambiguïté massive. La suite de « vient de __ » est structurellement très ouverte dans la langue : “loin”, “Rome”, “la mer”, “bon cœur”, “ce pays”, etc. Dans un petit corpus de ~3 000 proverbes, un grand nombre de ces continuations possibles n’apparaissent soit qu’une fois, soit pas du tout. Les comptes sont faibles, donc les estimations de probabilité conditionnelle deviennent instables ; et Laplace, en “comblant” les cases vides, réduit les écarts au profit de mots globalement fréquents. Ensuite, et surtout, le bigramme ne “voit” pas ce qui se passe deux mots en arrière. Or, de nombreuses terminaisons proverbiales sont des collocations de longueur au moins trois (“fait le larron”, “dans la mer”, “par la tête”). Avec seulement un mot d’historique, le modèle ne peut pas distinguer « vient de __ » dans le proverbe “a beau mentir qui vient de loin” d’un autre « vient de ___ » quelconque. Il bascule alors vers des bigrammes fréquents mais hors-sens (“t’ homme”, “que par”, “le monde”), parce que ce sont ceux dont les comptes sont les moins rares dans un si petit corpus. Autrement dit, l’incertitude contextuelle et la sparsité des observations entraînent le bigramme à “se rabattre” sur des solutions faciles et fréquentes, qui donnent mécaniquement des mots inadaptés.

- Le trigramme change la donne parce qu’il conditionne sur deux mots de contexte. Ce simple pas supplémentaire suffit à réactiver un grand nombre de patrons proverbiaux figés. Dès que l’historique contient « fait le », « prévenir que », « dans la », « par la », « sont deux », le modèle retrouve une continuité très fortement corrélée à la bonne réponse. On le voit dans les reconstructions : “le larron”, “que guérir”, “dans la mer”, “par la tête”, “sont deux” redeviennent probables. La stratégie de décodage utilisée – une petite recherche en faisceau suivie d’un re-classement final par perplexité de la phrase complète – consolide encore ce signal : plutôt que de s’enfermer dans un choix local myope, on évalue la fluidité globale, avec le padding BOS/EOS et les n-grammes de la phrase entière. C’est précisément ce qui permet au trigramme de dépasser la barre symbolique d’une phrase sur deux parfaitement reconstruite.

Pour autant, le trigramme n’est pas parfait, et là encore, la cause est structurelle. Avec un corpus de taille modeste, beaucoup de trigrammes décisifs n’apparaissent qu’une poignée de fois, voire jamais. Le lissage de Laplace évite les probabilités nulles mais tend à “sur-lisser” : en mettant un peu de masse partout, il atténue la force des collocations rares pourtant hautement discriminantes en fin de proverbe. Le modèle hésite alors entre des fins proches sémantiquement mais non proverbiales, par exemple « mentir » au lieu de « nuire », « main » au lieu de « mer », « fait » au lieu de « croire ». Ce ne sont pas des “erreurs absurdes”, ce sont des glissements typiques d’un modèle local peu informé au-delà de deux mots, confronté à une forte sparsité et à un lissage uniforme. Notre décodage aide, mais il ne peut pas compenser les limites intrinsèques du modèle : s’il manque d’évidence statistique, optimiser la recherche ne créera pas de signal.

Enfin, quelques remarques sur les métriques éclairent la lecture des scores. L’exact match au niveau de la phrase est volontairement strict : une seule erreur à une position masquée invalide l’ensemble, ce qui explique le faible rendement des N=1 et N=2 et le plafond du N=3. La mesure “accuracy sur mots masqués” isole la capacité locale à choisir le bon mot ; c’est là que l’on voit le trigramme culminer autour des deux tiers : lorsque le contexte de deux mots porte suffisamment d’information, la suite proverbiale redevient la plus probable ; mais dès que l’on sort des patrons fréquents, la sparsité et le lissage reprennent le dessus.

En somme, les “mots absurdes" en unigramme et, dans une moindre mesure, en bigramme ne sont pas un symptôme d’implémentation défaillante : ils sont la manifestation attendue d’un modèle qui, faute de contexte ou avec un contexte trop court, retombe sur le biais de fréquence globale et des bigrammes génériques. Le trigramme franchit un cap parce qu’il recolle aux collocations figées qui caractérisent la langue des proverbes ; mais il reste borné par la taille réduite du corpus et par un lissage (Laplace) qui aplatit les contrastes. LA longueur d’historique est le levier déterminant ; la sparsité et le lissage expliquent les plafonds ; et le décodage par beam + re-ranking à la perplexité améliore sensiblement N=3 sans pouvoir transcender les limites statistiques des modèles.


## Section 5 - Partie réservée pour faire nos tests lors de la correction

Merci de ne pas modifier ni retirer cette section du notebook.  